# Análisis Exploratorio Simple

In [28]:
import pandas as pd
import numpy as np
import requests

## 1. Petróleo Brent

In [29]:
df_brent = pd.read_csv("../data/raw/brent_prices.csv")

In [30]:
df_brent.head()

,date,brent_price
0,2025-01-02,75.930000
1,2025-01-03,76.510002
2,2025-01-06,76.300003
3,2025-01-07,77.050003
4,2025-01-08,76.160004


In [31]:
df_brent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224 entries, 0 to 223
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         224 non-null    object 
 1   brent_price  224 non-null    float64
dtypes: float64(1), object(1)
memory usage: 3.6+ KB


La variable "date" que corresponde a una fecha no trae el formato correcto.

In [32]:
# Convertir a datetime
df_brent['date'] = pd.to_datetime(df_brent['date'], format='%Y-%m-%d')

In [33]:
# Analizar si las fechas están completas

## 1. Rango de fechas
print("Fecha mínima:", df_brent['date'].min())
print("Fecha máxima:", df_brent['date'].max())
print("Total de registros:", len(df_brent))

## 2. Rango de fechas esperado vs real
fecha_min = df_brent['date'].min()
fecha_max = df_brent['date'].max()
rango_completo = pd.date_range(start=fecha_min, end=fecha_max, freq='B')

print("Fechas esperadas:", len(rango_completo))
print("Fechas reales:", df_brent['date'].nunique())


Fecha mínima: 2025-01-02 00:00:00
Fecha máxima: 2025-11-19 00:00:00
Total de registros: 224
Fechas esperadas: 230
Fechas reales: 224


In [34]:
# 3. Identificar fechas faltantes
fechas_faltantes = rango_completo.difference(df_brent['date'])
print("Fechas faltantes:", len(fechas_faltantes))

if len(fechas_faltantes) > 0:
    print(f"\nPrimeras {min(10, len(fechas_faltantes))} fechas faltantes:")
    print(fechas_faltantes[:10])
    
    # Analizar si son festivos o días especiales
    print("\nDías de la semana de fechas faltantes:")
    print(pd.Series(fechas_faltantes).dt.day_name().value_counts())
else:
    print("\nNo hay fechas faltantes - Serie completa para días hábiles")


Fechas faltantes: 6

Primeras 6 fechas faltantes:
DatetimeIndex(['2025-01-20', '2025-02-17', '2025-04-18', '2025-05-26',
               '2025-06-19', '2025-09-01'],
              dtype='datetime64[ns]', freq=None)

Días de la semana de fechas faltantes:
Monday      4
Friday      1
Thursday    1
Name: count, dtype: int64


In [35]:
# 4. Verificar duplicados
duplicados = df_brent['date'].duplicated().sum()
print(f"Fechas duplicadas: {duplicados}")

Fechas duplicadas: 0


In [36]:
# 5. Verificar continuidad (diferencias entre fechas consecutivas)
df_sorted = df_brent.sort_values('date').reset_index(drop=True)
df_sorted['dias_diff'] = df_sorted['date'].diff().dt.days

print("Distribución de diferencias entre fechas consecutivas (en días):")
print(df_sorted['dias_diff'].value_counts().sort_index())

Distribución de diferencias entre fechas consecutivas (en días):
dias_diff
1.0    176
2.0      1
3.0     41
4.0      5
Name: count, dtype: int64


Las fechas faltantes suelen corresponder a festivos bursátiles (Navidad, Año Nuevo, etc.), lo cual es normal para datos financieros.

In [38]:
# Agregación mensual
df_brent["year"] = df_brent['date'].dt.year
df_brent["month"] = df_brent['date'].dt.month

df_monthly = df_brent.groupby(['year', 'month']).agg({
    'brent_price' : [
        ('brent_price_mean', 'mean'),
        ('brent_price_min', 'min'),
        ('brent_price_max', 'max'),
        ('brent_price_median', 'median')
    ]
}).reset_index()

df_monthly.columns = ['year', 'month', 'brent_price_mean', 'brent_price_min', 'brent_price_max', 'brent_price_median']

# Redondear precios a 2 decimales
df_monthly[['brent_price_mean', 'brent_price_min', 'brent_price_max', 'brent_price_median']] = df_monthly[['brent_price_mean', 'brent_price_min', 'brent_price_max', 'brent_price_median']].round(2)
df_monthly.head()

,year,month,brent_price_mean,brent_price_min,brent_price_max,brent_price_median
0,2025,1,78.26,75.93,82.03,77.49
1,2025,2,74.94,72.53,77.00,74.78
2,2025,3,71.47,69.28,74.74,71.04
3,2025,4,66.46,62.82,74.95,65.85
4,2025,5,63.97,60.23,66.63,64.44


## 2. Datos de precios de la Secretaría de Energía de la Nación

Los archivos descargados de la Secretaría de Energía de la Nación contienen un registro detallado de precios de combustibles en estaciones de servicio de Argentina.

In [39]:
df_se = pd.read_csv("../data/raw/precios_eess_completo.csv")
df_se.head()

,Período,Operador,Nro Inscripción,Bandera,Fecha de baja,CUIT,Tipo Negocio,Dirección,Localidad,Provincia,...,Precio surtidor,NO Movimientos,Excentos,Impuesto Combustible Líquidos,Impuesto Dióxido Carbono,Tasa Vial,tasa Municipal,Ingresos Brutos,Iva,Fondo fiduciario GNC
0,2025/01,10 DE SETIEMBRE S.A.,1376,PUMA,NaN,33-64337382-9,Bocas de expendio (venta por menor) Duales (lí...,Av. Mosconi 299,LOMAS DEL MIRADOR,BUENOS AIRES,...,1.18,NO,0.0,20.12,2.29,1.5,0.01,3.06,21.0,0.00
1,2025/01,10 DE SETIEMBRE S.A.,1376,PUMA,NaN,33-64337382-9,Bocas de expendio (venta por menor) Duales (lí...,Av. Mosconi 299,LOMAS DEL MIRADOR,BUENOS AIRES,...,1450.00,NO,0.0,16.62,1.89,1.5,0.01,2.96,21.0,0.00
2,2025/01,10 DE SETIEMBRE S.A.,1376,PUMA,NaN,33-64337382-9,Bocas de expendio (venta por menor) Duales (lí...,Av. Mosconi 299,LOMAS DEL MIRADOR,BUENOS AIRES,...,464.90,NO,0.0,0.00,0.00,1.0,0.00,0.00,21.0,0.04
3,2025/01,10 DE SETIEMBRE S.A.,1376,PUMA,NaN,33-64337382-9,Bocas de expendio (venta por menor) Duales (lí...,Av. Mosconi 299,LOMAS DEL MIRADOR,BUENOS AIRES,...,1217.00,NO,0.0,29.59,1.81,1.5,0.01,3.29,21.0,0.00
4,2025/01,10 DE SETIEMBRE S.A.,1376,PUMA,NaN,33-64337382-9,Bocas de expendio (venta por menor) Duales (lí...,Av. Mosconi 299,LOMAS DEL MIRADOR,BUENOS AIRES,...,1488.00,NO,0.0,23.46,1.44,1.5,0.01,3.12,21.0,0.00


In [40]:
df_se["Producto"].value_counts()

Producto
Gas Oil Grado 2                     53318
Gas Oil Grado 3                     48893
Nafta (súper) entre 92 y 95 Ron     45464
Nafta (premium) de más de 95 Ron    42460
GNC                                 19432
N/D                                  1833
Kerosene                             1499
Nafta (común) hasta 92 Ron            492
GLPA                                   26
Name: count, dtype: int64

De este dataframe nos interesa quedarnos con datos de precios de combustibles directo a consumidor final.

In [41]:
# Limpieza y filtrado inicial

## 1. Normalización de nombres de columnas
df_se.columns = df_se.columns.str.lower().str.replace(' ', '_').str.replace(".", "_")
df_se.columns = df_se.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

## 2. Corrección de tipo de dato
df_se["periodo"] = pd.to_datetime(df_se["periodo"], format="%Y/%m", errors='coerce')

## 3. Normalización nombre de productos
PRODUCTO_MAP = {
    "nafta (super) entre 92 y 95 ron": "NAFTA GRADO 2",
    "nafta (premium) de más de 95 ron": "NAFTA GRADO 3",
    "nafta (común) hasta 92 ron": "NAFTA GRADO 1",
    "gas oil grado 2": "GASOIL GRADO 2",
    "gas oil grado 3": "GASOIL GRADO 3",
    "gnc" : "GNC",
    "kerosene": "KEROSENE",
    "glpa": "GLPA",
    "n/d": np.nan
}

df_se["producto"] = df_se["producto"].str.lower().map(PRODUCTO_MAP)

In [42]:
print("Dimensiones del dataframe:", df_se.shape)

Dimensiones del dataframe: (213417, 25)


In [43]:
df_se.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213417 entries, 0 to 213416
Data columns (total 25 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   periodo                        213417 non-null  datetime64[ns]
 1   operador                       213417 non-null  object        
 2   nro_inscripcion                213417 non-null  int64         
 3   bandera                        213417 non-null  object        
 4   fecha_de_baja                  3299 non-null    object        
 5   cuit                           213417 non-null  object        
 6   tipo_negocio                   213417 non-null  object        
 7   direccion                      213417 non-null  object        
 8   localidad                      213417 non-null  object        
 9   provincia                      213417 non-null  object        
 10  producto                       166120 non-null  object        
 11  

In [44]:
# Cantidad de duplicados
duplicados_se = df_se.duplicated().sum()
print(f"Cantidad de filas duplicadas: {duplicados_se}")

Cantidad de filas duplicadas: 7


In [45]:
df_se = df_se.drop_duplicates().reset_index(drop=True)

In [46]:
# Seleccion de columnas relevantes
columnas_relevantes = ["periodo", "provincia", "bandera", "producto", "precio_surtidor", "volumen"]
df_se_filtered = df_se[columnas_relevantes]
df_se_filtered.head()

,periodo,provincia,bandera,producto,precio_surtidor,volumen
0,2025-01-01,BUENOS AIRES,PUMA,GASOIL GRADO 2,1.18,24.11
1,2025-01-01,BUENOS AIRES,PUMA,GASOIL GRADO 3,1450.00,8.72
2,2025-01-01,BUENOS AIRES,PUMA,GNC,464.90,41164.80
3,2025-01-01,BUENOS AIRES,PUMA,NaN,1217.00,12.82
4,2025-01-01,BUENOS AIRES,PUMA,NAFTA GRADO 3,1488.00,2.52


In [47]:
df_se_filtered = df_se_filtered.dropna(subset="producto")

In [48]:
# Agregaciones básicas por periodo, provincia, bandera y producto
agrupado = df_se_filtered.groupby(["periodo", "provincia", "bandera", "producto"]).agg(
    precio_surtidor_mediana=("precio_surtidor", "median"),
    volumen_total=("volumen", "sum")
)
agrupado = agrupado.reset_index()
agrupado.head()

,periodo,provincia,bandera,producto,precio_surtidor_mediana,volumen_total
0,2025-01-01,BUENOS AIRES,AXION,GASOIL GRADO 2,1246.0,162636.25
1,2025-01-01,BUENOS AIRES,AXION,GASOIL GRADO 3,1468.0,150130.98
2,2025-01-01,BUENOS AIRES,AXION,GNC,559.0,7810727.58
3,2025-01-01,BUENOS AIRES,AXION,KEROSENE,1627.5,2.77
4,2025-01-01,BUENOS AIRES,AXION,NAFTA GRADO 3,1505.0,209109.22


In [49]:
agrupado["producto"].value_counts()

producto
GASOIL GRADO 2    1568
GASOIL GRADO 3    1564
NAFTA GRADO 3     1522
GNC               1083
KEROSENE           379
NAFTA GRADO 1      275
GLPA                26
Name: count, dtype: int64

In [50]:
agrupado.isna().sum()

periodo                    0
provincia                  0
bandera                    0
producto                   0
precio_surtidor_mediana    0
volumen_total              0
dtype: int64

## 3. Datos de dolar blue y oficial

In [13]:
url = "https://api.bluelytics.com.ar/v2/evolution.json"

response = requests.get(url, timeout = 30)
response.raise_for_status()

data = response.json()

In [51]:
df_dolar = pd.DataFrame(data)
df_dolar.head()

,date,source,value_sell,value_buy
0,2025-11-14,Oficial,1433.0,1382.0
1,2025-11-14,Blue,1430.0,1410.0
2,2025-11-13,Oficial,1434.0,1383.0
3,2025-11-13,Blue,1435.0,1415.0
4,2025-11-12,Oficial,1444.0,1393.0


In [52]:
df_dolar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8835 entries, 0 to 8834
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        8835 non-null   object 
 1   source      8835 non-null   object 
 2   value_sell  8835 non-null   float64
 3   value_buy   8835 non-null   float64
dtypes: float64(2), object(2)
memory usage: 276.2+ KB


In [53]:
df_dolar["date"] = pd.to_datetime(df_dolar["date"], format="%Y-%m-%d")

In [54]:
df_dolar["date"].min(), df_dolar["date"].max()

(Timestamp('2011-01-03 00:00:00'), Timestamp('2025-11-14 00:00:00'))

In [55]:
# Filtro desde 2025-01-01
df_dolar = df_dolar[df_dolar["date"] >= "2025-01-01"].reset_index(drop=True)
df_dolar.head()

,date,source,value_sell,value_buy
0,2025-11-14,Oficial,1433.0,1382.0
1,2025-11-14,Blue,1430.0,1410.0
2,2025-11-13,Oficial,1434.0,1383.0
3,2025-11-13,Blue,1435.0,1415.0
4,2025-11-12,Oficial,1444.0,1393.0


In [56]:
df_dolar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456 entries, 0 to 455
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        456 non-null    datetime64[ns]
 1   source      456 non-null    object        
 2   value_sell  456 non-null    float64       
 3   value_buy   456 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 14.4+ KB
